In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
%%capture
!pip install gradio

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "Minervus00/Llama-3.2-3B-text-to-quiz-json",
    model_name = "Minervus00/mistral-7b-text-to-quiz-json",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
_ = FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.9: Fast Mistral patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.11.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

instruction = 'Given the following text paragraph, generate a quiz question in {} with four options and specify the correct answer  and an explanation in the format: {{"question": "...", "options": ["...", "...", "...", "..."], "answer": "A" or "B" or "C" or "D", "explanation": "..."}}.'


In [ ]:
import json

def text_to_quiz(input_data):
  input_data = json.loads(input_data)
  text = input_data["text"]
  lang = input_data["lang"]
  inputs = tokenizer(
  [
      prompt.format(
          instruction.format(lang), # instruction
          text,
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt")

  # Generate the output
  generated_tokens = model.generate(**inputs, max_new_tokens=256)

  # Decode the generated tokens into text
  generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

  # Remove any trailing "<|eot_id>" token if it's still there
  # cleaned_text = generated_text.replace("<|eot_id>", "").strip()
  cleaned_text = generated_text[generated_text.find("### Response:\n") + 14:]

  return {"output": cleaned_text}

In [ ]:
# data = {"text": """Jacques Foccart, né en 1913 et décédé en 1997, était un homme politique et conseiller français proche du général Charles de Gaulle. Il est surtout connu pour son rôle de conseiller aux affaires africaines et malgaches sous les présidences de De Gaulle et de Georges Pompidou. Foccart a joué un rôle clé dans la politique africaine de la France, souvent appelée "Françafrique", en maintenant des relations étroites avec les dirigeants africains et en influençant les politiques françaises sur le continent. Il a également été impliqué dans diverses opérations secrètes et a été une figure controversée en raison de son influence et de ses méthodes. Foccart a laissé une empreinte durable sur les relations franco-africaines et est souvent considéré comme l'architecte de la politique africaine de la France pendant la période gaulliste.""",
#         "lang": "ENGLISH"}

In [ ]:
# resp = text_to_quiz(json.dumps(data))
# resp["output"]

In [ ]:
# print(resp["output"])

In [ ]:
import gradio as gr

demo = gr.Interface(
    fn=text_to_quiz,
    inputs="json",
    outputs="json"
)

demo.queue().launch(share=True, debug=True, show_error=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6329753acc4c323e75.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6329753acc4c323e75.gradio.live
